<a href="https://colab.research.google.com/github/Rod10nN/rps/blob/master/dz3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random
import random
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Loading environment lux_ai_s3 failed: No module named 'gymnax'


### 1. Агент "камень": всегда выбирает "камень".

In [3]:
%%writefile agent_rock.py

def agent_rock(observation, configuration):
    return 0 # 0 - камень

Writing agent_rock.py


### 2. Агент "бумага": всегда выбирает "бумагу".

In [4]:
%%writefile agent_paper.py

def agent_paper(observation, configuration):
    return 1  # 1 - бумага


Writing agent_paper.py


### 3. Агент "ножницы": всегда выбирает "ножницы".

In [5]:
%%writefile agent_scissors.py

def agent_scissors(observation, configuration):
    return 2 # 2 - ножницы

Writing agent_scissors.py


### 4. Случайный агент: выбирает случайное действие.

In [6]:
%%writefile agent_random.py

def agent_random(observation, configuration):
    return random.randrange(0, configuration.signs)


Writing agent_random.py


### 5. Копирующий агент: копирует последнее действие противника.

In [7]:
%%writefile copy_opponent.py

def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


### 6. Агент "чередующийся": чередует между "камень", "ножницы" и "бумага".

In [8]:
%%writefile agent_alternating.py

def agent_alternating(observation, configuration):
    return observation.step % 3  # Чередуем: 0, 1, 2 (камень, бумага, ножницы)

Writing agent_alternating.py


### 7. Агент "победитель": всегда повторяет действие победителя последнего раунда.

In [9]:
%%writefile agent_winner.py


def agent_winner(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    return observation.lastOpponentAction if observation.result == 1 else observation.lastOpponentAction  # повторяет победившее действие


Writing agent_winner.py


### 8. Агент "агрессивный ответ": выбирает действие, выигрывающее против последнего действия противника.

In [10]:
%%writefile agent_aggressive_counter.py

def agent_aggressive_counter(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    # выбирает действие, которое побеждает противника
    return (observation.lastOpponentAction + 1) % 3  # против "камня" - "бумага", против "бумаги" - "ножницы", против "ножниц" - "камень"


Writing agent_aggressive_counter.py


### 9. Агент "обратный": выбирает действие, проигрывающее против последнего действия противника

In [11]:
%%writefile agent_reverse.py

def agent_reverse(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    # выбирает действие, которое проигрывает противнику
    return (observation.lastOpponentAction + 2) % 3  # против "камня" - "ножницы", против "бумаги" - "камень", против "ножниц" - "бумага"


Writing agent_reverse.py


### Соревнование между агентами

In [14]:
agents = [
    "agent_rock.py",
    "agent_paper.py",
    "agent_scissors.py",
    "agent_random.py",
    "copy_opponent.py",
    "agent_alternating.py",
    "agent_winner.py",
    "agent_aggressive_counter.py",
    "agent_reverse.py",
]
for i in range(len(agents)): # все агенты должны сыграть друг с другом
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]
        print(f'{agent1}  {evaluate("rps",[agent1, agent2],configuration={"episodeSteps": 100})} {agent2}')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


agent_rock.py  [[-99.0, 99.0]] agent_paper.py
agent_rock.py  [[99.0, -99.0]] agent_scissors.py
agent_rock.py  [[1, None]] agent_random.py
agent_rock.py  [[1, None]] copy_opponent.py
agent_rock.py  [[0, 0]] agent_alternating.py
agent_rock.py  [[1, None]] agent_winner.py
agent_rock.py  [[1, None]] agent_aggressive_counter.py
agent_rock.py  [[1, None]] agent_reverse.py
agent_paper.py  [[-99.0, 99.0]] agent_scissors.py
agent_paper.py  [[1, None]] agent_random.py
agent_paper.py  [[1, None]] copy_opponent.py
agent_paper.py  [[0, 0]] agent_alternating.py
agent_paper.py  [[1, None]] agent_winner.py
agent_paper.py  [[1, None]] agent_aggressive_counter.py
agent_paper.py  [[1, None]] agent_reverse.py
agent_scissors.py  [[1, None]] agent_random.py
agent_scissors.py  [[1, None]] copy_opponent.py
agent_scissors.py  [[0, 0]] agent_alternating.py
agent_scissors.py  [[1, None]] agent_winner.py
agent_scissors.py  [[1, None]] agent_aggressive_counter.py
agent_scissors.py  [[1, None]] agent_reverse.py
age

Больще всего побед одержала стратегия выбирать только камень *agent_rock.py* с 6 победами